In [1]:
import pandas
import openai
import requests
import time

In [2]:
# Function to load the API key from a file
def load_api_key(api_key_file_path):
    with open(api_key_file_path, 'r') as file:
        api_key = file.read().strip()
    return api_key

# Load your API key from a file
api_key = load_api_key('api-key.txt')

# Set up the OpenAI API client
openai.api_key = api_key

github_pat = load_api_key('github-pat.txt')
headers = {'Authorization': 'token ' + github_pat}

In [15]:
def get_chat_response(user_prompt, system_message, model="gpt-3.5-turbo-16k", max_tokens=1000):
    response = openai.ChatCompletion.create(
        model=model,
        messages=[{"role": "system", "content": system_message},
                  {"role": "user", "content": user_prompt}],
        max_tokens=max_tokens,
        temperature = 0.2
    )
    return response.choices[0].message['content'].strip()

In [4]:
def generate_build_pipeline(repo_structure, languages, dependencies):
    system_message = "Your name is Dev bot. You are a brilliant and meticulous engineer assigned to write a GitHub Actions workflow in YAML for the following Github Repository. When you write code, the code works on the first try, is syntactically perfect and is fully complete. The workflow should be able to build and run the application and run the tests if present in the repository. Take into account the current repository's language, frameworks, and dependencies. "

    user_prompt = """
    Analyze the github repository structure, language, framework and dependencies provide below to create a github action build workflow. You will provide the github action workflow as the answer. Only include the yaml file in the output. Do not add any other text before or after the code.
    
    Repository structure:
    {repo_structure}

    Languages: 
    {languages}

    Dependencies: 
    {dependencies}
    
    """

    request_input = {
        'repo_structure': repo_structure,
        'languages': languages,
        'dependencies': dependencies
    }

    user_prompt = user_prompt.format(**request_input)

    response = get_chat_response(user_prompt=user_prompt, system_message=system_message)

    return response.strip("```")

In [5]:
def get_repository_tree(repository_identifier, branch='main'):

    repository_tree = []
    response = requests.get(f'https://api.github.com/repos/{repository_identifier}/git/trees/{branch}', headers=headers)
    data = response.json()

    if response.status_code == 200:
        root_tree = data['tree']
        max_depth = 2

        def get_tree_recursive(tree_sha, current_depth):
            if current_depth > max_depth:
                return

            response = requests.get(f'https://api.github.com/repos/{repository_identifier}/git/trees/{tree_sha}', headers=headers)
            data = response.json()

            if response.status_code == 200:
                for item in data['tree']:
                    if item['type'] == 'tree':
                        repository_tree.append(f"Directory: {item['path']}")
                        get_tree_recursive(item['sha'], current_depth + 1)
                    else:
                        repository_tree.append(f"File: {item['path']}")

        get_tree_recursive(data['sha'], 1)
    else:
        print(f"Failed to fetch tree: {data['message']}")

    return repository_tree

In [6]:
def get_list_of_languages(repository_identifier):
    url = 'https://api.github.com/repos/' + repository_identifier + '/languages'

    response = requests.get(url, headers=headers)
    
    return response.json().keys()

In [7]:
def get_list_of_dependencies(repository_identifier):
    url = 'https://api.github.com/repos/' + repository_identifier + '/dependency-graph/sbom'

    response = requests.get(url, headers=headers)
    response.json()['sbom']['packages']
    dependency_names = [package['name'] + ', version = ' + package['versionInfo'] for package in response.json()['sbom']['packages']]
    
    return dependency_names


In [8]:
def get_default_branch(repository_identifier):
    url = 'https://api.github.com/repos/' + repository_identifier

    response = requests.get(url, headers=headers)
    
    return response.json()['default_branch']

In [13]:
def run_experiment(csvFile):
    for i in range(0,len(csvFile)):
        repo_identifier = csvFile.iloc[i]['GitHub_Repo_Link'].split('github.com/')[1]
        print(repo_identifier)
        try:
            repo_structure = get_repository_tree(repo_identifier, get_default_branch(repo_identifier))
            languages = get_list_of_languages(repo_identifier)
            dependencies = get_list_of_dependencies(repo_identifier)
            csvFile.loc[i,'Generated_Build_Pipeline_File_Content'] = generate_build_pipeline(repo_structure, languages, dependencies)
            # Add delay to avoid rate limiting
            time.sleep(30)
        except Exception as e:
            print(e)
            continue

    csvFile.to_csv('DevOps_LLM_Bot_Test_Data - C#.csv', index=False)

In [12]:
# reading the CSV file
csvFile = pandas.read_csv('DevOps_LLM_Bot_Test_Data - C#.csv')

csvFile = csvFile[csvFile['GitHub_Repo_Link'].notna()]
csvFile

,Index,GitHub_Repo_Link,GitHub_Build_Pipeline_File_Link,GitHub_Build_Pipeline_File_Content,Generated_Build_Pipeline_File_Content,Result
0,1.0,https://github.com/DapperLib/Dapper,https://github.com/DapperLib/Dapper/blob/main/...,name: Main Build\n\non:\n pull_request:\n pu...,yaml\nname: Build and Test\n\non:\n push:\n ...,NaN
1,2.0,https://github.com/huiyadanli/RevokeMsgPatcher,https://github.com/huiyadanli/RevokeMsgPatcher...,name: .Net Build\n\non:\n workflow_dispatch:\...,yaml\nname: Build and Test\n\non:\n push:\n ...,NaN
2,3.0,https://github.com/jasontaylordev/CleanArchite...,https://github.com/jasontaylordev/CleanArchite...,name: Build\n\non:\n pull_request:\n branc...,NaN,NaN
3,4.0,https://github.com/nilaoda/N_m3u8DL-CLI,https://github.com/nilaoda/N_m3u8DL-CLI/blob/m...,name: Build_Latest\n \non: [push]\n \njobs:\n ...,yaml\nname: Build and Test\n\non:\n push:\n ...,NaN
4,5.0,https://github.com/shadowsocks/shadowsocks-win...,https://github.com/shadowsocks/shadowsocks-win...,name: Build\n\non:\n push:\n branches-igno...,yaml\nname: Build and Test\n\non:\n push:\n ...,NaN
5,6.0,https://github.com/ShareX/ShareX,https://github.com/ShareX/ShareX/blob/develop/...,name: Build ShareX\n\non:\n push:\n branch...,yaml\nname: Build and Test\n\non:\n push:\n ...,NaN
6,7.0,https://github.com/DapperLib/Dapper,https://github.com/DapperLib/Dapper/blob/main/...,name: Main Build\n\non:\n pull_request:\n pu...,yaml\nname: Build and Test\n\non:\n push:\n ...,NaN
7,8.0,https://github.com/aalhour/C-Sharp-Algorithms,https://github.com/aalhour/C-Sharp-Algorithms/...,name: Build and Test\n\non: [push]\n\njobs:\n ...,yaml\nname: Build and Test\n\non:\n push:\n ...,NaN
8,9.0,https://github.com/Cysharp/UniTask,https://github.com/Cysharp/UniTask/blob/master...,name: build-release\n\non:\n workflow_dispatc...,yaml\nname: Build and Test\n\non:\n push:\n ...,NaN
9,10.0,https://github.com/EduardoPires/EquinoxProject,https://github.com/EduardoPires/EquinoxProject...,name: .NET Core\n\non:\n push:\n branches:...,yaml\nname: Build and Test\n\non:\n push:\n ...,NaN


In [16]:
run_experiment(csvFile)

MassTransit/MassTransit
This model's maximum context length is 16385 tokens. However, your messages resulted in 39191 tokens. Please reduce the length of the messages.
